# Save DTA Draft

Creates or updates draft version for DTA(s) in Silver table.

**Operation:** `SAVE_DRAFT`

**Modes:**
- `HISTORICAL`: Links existing silver records to DTA with draft version
- `UI`: Saves user edits as new draft (increments draft version)

**What it does:**
1. Gets DTA ID(s) from widget or task values (from create_dta_instance)
2. Updates Silver records with `dta_id`, `version_tag`, `version_status=DRAFT`
3. Registers draft version in `md_version_registry`
4. Updates DTA entity with `current_draft_version`

**Parameters:**
- `dta_id` - Single DTA identifier (for UI/API call)
- `source` - HISTORICAL or UI
- `library_type` - Library type (e.g., transfer_variables)


In [ ]:
# Cell 1: Imports
import json
from datetime import datetime
from pyspark.sql import functions as F
from clinical_data_standards_framework.versioning import get_version_registry_schema

print("="*80)
print("SAVE DTA DRAFT")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
silver_schema = globals_dict['silver_schema']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

print(f"Catalog: {catalog}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters and DTA IDs
dbutils.widgets.text("dta_id", "", "DTA ID (single, for UI)")
dbutils.widgets.text("source", "UI", "Source (HISTORICAL or UI)")
dbutils.widgets.text("library_type", "", "Library Type")

dta_id_param = dbutils.widgets.get("dta_id")
source = dbutils.widgets.get("source")
library_type_param = dbutils.widgets.get("library_type")

# Determine library type
if library_type_param:
    library_type = library_type_param
elif library_tables:
    library_type = library_tables[0].get('library_type', 'transfer_variables')
else:
    library_type = 'transfer_variables'

# Get DTA IDs: Priority order:
# 1. Widget parameter (for UI/API calls via version manager)
# 2. Task values from create_dta_instance (for HISTORICAL flow in job_cdm_dta_create)
dta_ids = []
dta_numbers = []

if dta_id_param:
    # Called via version manager or API with explicit dta_id
    dta_ids = [dta_id_param]
    print(f"Mode: Single DTA from widget parameter")
else:
    # Try to get from create_dta_instance task values (HISTORICAL flow)
    try:
        dta_ids = json.loads(dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="created_dta_ids"))
        dta_numbers = json.loads(dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="created_dta_numbers"))
        print(f"Mode: Multiple DTAs from create_dta_instance task values")
    except Exception:
        # No task values - this is expected when called via version manager
        pass

# Validate we have at least one DTA ID
if not dta_ids:
    raise ValueError(
        "No dta_id provided. When calling SAVE_DRAFT via job_cdm_version_manager, "
        "you must provide the dta_id parameter."
    )

print(f"\nSource: {source}")
print(f"Library Type: {library_type}")
print(f"DTAs to process: {len(dta_ids)}")


In [ ]:
# Cell 4: Configure Table Names
silver_table = f"{catalog}.{silver_schema}.md_transfer_variable_field_normalized"
dta_table = f"{catalog}.{gold_schema}.dta"
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

print(f"\nSilver Table: {silver_table}")
print(f"DTA Table: {dta_table}")
print(f"Registry Table: {registry_table}")


In [ ]:
# Cell 5: Load DTA Metadata
df_dta = spark.table(dta_table).filter(F.col("dta_id").isin(dta_ids))
dta_metadata = {row["dta_id"]: row.asDict() for row in df_dta.collect()}

# If dta_numbers weren't provided, look them up
if not dta_numbers:
    dta_numbers = [dta_metadata[dta_id]["dta_number"] for dta_id in dta_ids]
    print(f"Looked up DTA numbers: {dta_numbers}")

print(f"\nLoaded metadata for {len(dta_metadata)} DTAs")


In [ ]:
# Cell 6: Create Draft Versions in Silver
now = datetime.now()
base_library_version = "1.0"  # TODO: Get from config or DTA

print(f"\n{'='*80}")
print("Creating draft versions in Silver...")
print(f"{'='*80}")

draft_versions = {}

for idx, dta_id in enumerate(dta_ids):
    meta = dta_metadata.get(dta_id, {})
    dta_number = meta.get("dta_number") or dta_numbers[idx]
    parent_document_id = meta.get("parent_document_id")
    
    # Create draft version tag
    draft_version_tag = f"{base_library_version}-{dta_number}-draft1"
    draft_versions[dta_id] = draft_version_tag
    
    print(f"\nProcessing DTA: {dta_number} (ID: {dta_id[:8]}...)")
    print(f"  Draft version tag: {draft_version_tag}")
    
    # For HISTORICAL: Update silver records by parent_document_id
    # For UI: Records already have dta_id (from branch or previous draft)
    if source == "HISTORICAL" and parent_document_id:
        update_sql = f"""
            UPDATE {silver_table}
            SET dta_id = '{dta_id}',
                version_tag = '{draft_version_tag}',
                version_status = 'DRAFT',
                is_current_draft = true
            WHERE parent_document_id = '{parent_document_id}'
              AND (dta_id IS NULL OR dta_id = '')
        """
        spark.sql(update_sql)
        
        # Count updated records
        updated_count = spark.table(silver_table).filter(
            (F.col("dta_id") == dta_id) & 
            (F.col("version_tag") == draft_version_tag)
        ).count()
        print(f"  Updated {updated_count} silver records")
    else:
        # UI mode - update by dta_id
        update_sql = f"""
            UPDATE {silver_table}
            SET version_tag = '{draft_version_tag}',
                version_status = 'DRAFT',
                is_current_draft = true
            WHERE dta_id = '{dta_id}'
        """
        spark.sql(update_sql)
        
        updated_count = spark.table(silver_table).filter(
            (F.col("dta_id") == dta_id) &
            (F.col("version_tag") == draft_version_tag)
        ).count()
        print(f"  Updated {updated_count} silver records")


In [ ]:
# Cell 7: Register Draft Versions in Registry and Update DTA Entities
print(f"\n{'='*80}")
print("Registering draft versions and updating DTA entities...")
print(f"{'='*80}")

# Ensure registry table exists
if not spark.catalog.tableExists(registry_table):
    print(f"Creating registry table: {registry_table}")
    empty_df = spark.createDataFrame([], schema=get_version_registry_schema())
    empty_df.write.format("delta").mode("overwrite").saveAsTable(registry_table)

registry_records = []
for dta_id, draft_version_tag in draft_versions.items():
    meta = dta_metadata.get(dta_id, {})
    
    # Count records for this draft
    record_count = spark.table(silver_table).filter(
        (F.col("dta_id") == dta_id) &
        (F.col("version_tag") == draft_version_tag)
    ).count()
    
    registry_record = {
        "version_tag": draft_version_tag,
        "library_type": library_type,
        "version_type": "DTA_DRAFT",
        "is_latest_major": False,
        "dta_id": dta_id,
        "dta_trial_id": meta.get("trial_id"),
        "dta_stream_type": meta.get("data_stream_type"),
        "dta_provider_name": meta.get("data_provider_name"),
        "parent_version": base_library_version,
        "record_count": record_count,
        "status": "ACTIVE",
        "notes": f"Draft created from {source}",
        "created_by_principal": created_by_principal,
        "created_ts": now,
        "last_updated_by_principal": created_by_principal,
        "last_updated_ts": now,
        "databricks_job_id": databricks_job_id,
        "databricks_job_name": databricks_job_name,
        "databricks_run_id": databricks_run_id
    }
    registry_records.append(registry_record)
    print(f"  Registered: {draft_version_tag} ({record_count} records)")
    
    # Update DTA entity
    spark.sql(f"""
        UPDATE {dta_table}
        SET current_version_tag = '{draft_version_tag}',
            current_draft_version = '{draft_version_tag}',
            base_library_version = '{base_library_version}',
            last_updated_by_principal = '{created_by_principal}',
            last_updated_ts = current_timestamp(),
            databricks_job_id = '{databricks_job_id}',
            databricks_run_id = '{databricks_run_id}'
        WHERE dta_id = '{dta_id}'
    """)
    print(f"  Updated DTA {meta.get('dta_number', dta_id[:8])} with draft version")

if registry_records:
    registry_df = spark.createDataFrame(registry_records, schema=get_version_registry_schema())
    registry_df.write.format("delta").mode("append").saveAsTable(registry_table)
    print(f"\nRegistered {len(registry_records)} draft version(s)")


In [ ]:
# Cell 8: Set Task Values and Summary
dbutils.jobs.taskValues.set(key="draft_versions", value=json.dumps(draft_versions))
dbutils.jobs.taskValues.set(key="dta_ids", value=json.dumps(dta_ids))
dbutils.jobs.taskValues.set(key="action", value="SAVE_DRAFT")

print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"Source: {source}")
print(f"DTAs processed: {len(draft_versions)}")
for dta_id, version in draft_versions.items():
    meta = dta_metadata.get(dta_id, {})
    print(f"  {meta.get('dta_number', dta_id[:8])}: {version}")
print(f"\n✅ Drafts saved successfully!")
